In [2]:
import json
import pandas as pd

In [3]:
with open('CAIC_Forecasts.json', 'r') as file:
    forecasts = json.load(file)

In [4]:
for forecast in forecasts:
    forecast.pop('polygons', None)
    
    summary_dict = json.loads(forecast['Summary'])
    forecast['Summary_en'] = summary_dict.get('en', None)

In [5]:
forecast_df = pd.DataFrame(forecasts)
forecast_df.head(5)

,id,Issued Date,date,Last Modified,Expiry,Day,Above Treeline,Treeline,Below Treeline,Summary,Summary_en
0,534290d5-888d-4f06-9be5-9f97ff325206,2023-11-01T13:15:59.054Z,2023-11-01,2023-11-01T13:20:57Z,2023-11-01T22:30:25.134Z,issueDate,low,low,low,"{""en"": """"}",
1,4ff007af-d822-46ae-b9fc-9ca0c1ccea89,2023-11-01T13:15:59.435Z,2023-11-01,2023-11-01T13:21:06Z,2023-11-01T22:30:00.000Z,issueDate,noForecast,noForecast,noForecast,"{""en"": """"}",
2,8d1f9204-2c90-40df-92c4-23760e375efd,2023-11-01T13:15:59.571Z,2023-11-01,2023-11-01T13:21:02Z,2023-11-01T22:30:18.525Z,issueDate,low,low,low,"{""en"": """"}",
3,a76838f4-11e7-48c4-850d-1b6ee3dd388d,2023-11-01T13:15:59.883Z,2023-11-01,2023-11-01T13:20:52Z,2023-11-01T22:30:02.651Z,issueDate,moderate,moderate,low,"{""en"": """"}",
4,3c9d3f41-454d-42e8-9f2c-aeb952aa463c,2023-11-01T19:45:59.379Z,2023-11-01,2023-11-01T19:57:06Z,2023-11-01T22:30:02.651Z,issueDate,moderate,moderate,low,"{""en"": """"}",


In [6]:
forecast_df.isna().sum()

id                0
Issued Date       0
date              0
Last Modified     0
Expiry            0
Day               0
Above Treeline    0
Treeline          0
Below Treeline    0
Summary           0
Summary_en        1
dtype: int64

In [7]:
forecast_df = forecast_df[forecast_df['Summary_en'] != ""]

In [8]:
len(forecast_df)

3334

In [10]:
forecast_df = forecast_df[(forecast_df['Below Treeline'] != "noRating") & (forecast_df['Treeline'] != "noRating") & (forecast_df['Above Treeline'] != "noRating")]

In [11]:
len(forecast_df)

3177

In [12]:
forecast_df['Above Treeline'].value_counts()

Above Treeline
moderate        1457
considerable     903
low              662
high             137
earlySeason       18
Name: count, dtype: int64

In [13]:
forecast_df.dtypes

id                object
Issued Date       object
date              object
Last Modified     object
Expiry            object
Day               object
Above Treeline    object
Treeline          object
Below Treeline    object
Summary           object
Summary_en        object
dtype: object

In [14]:
from bs4 import BeautifulSoup as bsoup
def strip_html(html):
    text = str(html)
    return bsoup(text, "html.parser").get_text()
    
forecast_df['Summary_clean'] = forecast_df['Summary_en'].apply(strip_html)

In [15]:
forecast_df.dtypes

id                object
Issued Date       object
date              object
Last Modified     object
Expiry            object
Day               object
Above Treeline    object
Treeline          object
Below Treeline    object
Summary           object
Summary_en        object
Summary_clean     object
dtype: object

In [16]:
forecast_df.shape

(3177, 12)

In [17]:
forecast_df = forecast_df[forecast_df['Summary_clean'] != ""]

In [18]:
forecast_df.shape

(2885, 12)

In [20]:
forecast_df["datetime"] = pd.to_datetime(forecast_df["date"])

In [21]:
forecast_df["datetime"].max()

Timestamp('2025-01-27 00:00:00')

In [23]:
forecast_df.dtypes

id                        object
Issued Date               object
date                      object
Last Modified             object
Expiry                    object
Day                       object
Above Treeline            object
Treeline                  object
Below Treeline            object
Summary                   object
Summary_en                object
Summary_clean             object
datetime          datetime64[ns]
dtype: object

In [24]:
forecast_df.head()

,id,Issued Date,date,Last Modified,Expiry,Day,Above Treeline,Treeline,Below Treeline,Summary,Summary_en,Summary_clean,datetime
5,0bb40c21-682a-4987-8a92-1ae9a4a00d40,2023-11-01T22:30:00Z,2023-11-01,2023-11-01T20:59:05Z,2023-11-02T22:30:00Z,issueDatePlusOne,low,low,low,"{""en"": ""<p>The only place where you can trigge...",<p>The only place where you can trigger an ava...,The only place where you can trigger an avalan...,2023-11-01
9,f2ffe07c-7cc2-4a22-a6b5-05cbca464ff1,2023-11-02T22:30:00Z,2023-11-02,2023-11-03T11:37:00Z,2023-11-03T22:30:00Z,issueDatePlusOne,moderate,moderate,low,"{""en"": ""<p>Avalanche conditions are most dange...",<p>Avalanche conditions are most dangerous on ...,Avalanche conditions are most dangerous on win...,2023-11-02
10,c86207e2-6a48-4f65-a449-68ea3f759ec7,2023-11-02T22:30:00Z,2023-11-02,2023-11-03T11:36:49Z,2023-11-03T22:30:00Z,issueDatePlusOne,low,low,low,"{""en"": ""<p>The areas where you may be most tem...",<p>The areas where you may be most tempted to ...,The areas where you may be most tempted to tra...,2023-11-02
11,d49d1461-8a31-4a16-a121-2a02da4c3af7,2023-11-02T22:30:00Z,2023-11-02,2023-11-03T11:36:40Z,2023-11-03T22:30:00Z,issueDatePlusOne,low,low,low,"{""en"": ""<p>Most slopes are free from avalanche...",<p>Most slopes are free from avalanches simply...,Most slopes are free from avalanches simply du...,2023-11-02
12,cae3c30e-42e1-40f7-8e1e-bdcef73c5d85,2023-11-02T22:30:00Z,2023-11-02,2023-11-02T22:51:18Z,2023-11-03T22:30:00Z,issueDatePlusOne,low,low,low,"{""en"": ""<p>The areas where you may be most tem...",<p>The areas where you may be most tempted to ...,The areas where you may be most tempted to tra...,2023-11-02


In [39]:
forecast_df.isna().sum()

id                0
Issued Date       0
date              0
Last Modified     0
Expiry            0
Day               0
Above Treeline    0
Treeline          0
Below Treeline    0
Summary           0
Summary_en        1
Summary_clean     0
datetime          0
dtype: int64

In [56]:
df = forecast_df[["datetime", "Above Treeline", "Treeline", "Below Treeline", "Summary_clean"]]

In [57]:
# pulling in manually pulled forecasts
with open('manual_forecasts.json', 'r') as f:
    manual_forecasts = json.load(f)

manual_forecast_df = pd.DataFrame(manual_forecasts)

In [58]:
manual_forecast_df["datetime"] = pd.to_datetime(manual_forecast_df["date"])

In [59]:
manual_forecast_df.isna().sum()

date              0
Above Treeline    0
Treeline          0
Below Treeline    0
Summary_clean     0
datetime          0
dtype: int64

In [60]:
manual_forecast_df.columns

Index(['date', 'Above Treeline', 'Treeline', 'Below Treeline', 'Summary_clean',
       'datetime'],
      dtype='object')

In [61]:
manual_forecast_df = manual_forecast_df.drop(columns=['date'])

In [62]:
manual_forecast_df.shape

(56, 5)

In [63]:
df.shape

(2885, 5)

In [64]:
manual_forecast_df.columns

Index(['Above Treeline', 'Treeline', 'Below Treeline', 'Summary_clean',
       'datetime'],
      dtype='object')

In [65]:
df.columns

Index(['datetime', 'Above Treeline', 'Treeline', 'Below Treeline',
       'Summary_clean'],
      dtype='object')

In [66]:
# concat dfs
df = pd.concat([df, manual_forecast_df], ignore_index=True)

In [67]:
df.shape

(2941, 5)

In [68]:
df.isna().sum()

datetime          0
Above Treeline    0
Treeline          0
Below Treeline    0
Summary_clean     0
dtype: int64

In [69]:
df.sample()

,datetime,Above Treeline,Treeline,Below Treeline,Summary_clean
1004,2024-02-23,moderate,moderate,moderate,Dangerous avalanche conditions exist where win...


In [70]:
# create new csv
df.to_csv("forecast_data.csv", index=False)